In [155]:
!blastn -task blastn -query data/res/sRNA_DEG.fasta  -subject data/data/miRNAs/mature.fa -outfmt "6 qseqid sseqid qstart qend sstart send score length mismatch gaps gapopen nident pident qlen slen qcovs" > data/res/sRNA_DEG_mirbase.blast.csv

In [156]:
import pandas as pd
import numpy as np
data_dir = 'data'
lb = 'iwgsc_10_12_18'
blast_out = data_dir + '/res/sRNA_DEG_mirbase.blast.csv'

In [157]:
deg_file = data_dir + '/res/deg_data.csv'
df_deg = pd.read_csv(deg_file,sep="\t")
df_deg.loc[pd.isnull(df_deg['MajorRNA']), 'MajorRNA'] = df_deg['Gene']
print('Res len:',len(df_deg.index))
df_deg.head(3)

Res len: 14769


,Unnamed: 0,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,...,DicerCall,MIRNA,PhaseScore,Short,Long,20,21,22,23,24
0,1,UUGUACCAACUGUAAACUCGU,2442.153702,11.330772,0.881097,12.859849,7.572419e-38,1.445802e-33,2.131286,1.558099,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,AGAUUCUGUUGAACUCGUUCUC,1771.326168,10.859947,0.862006,12.598466,2.152912e-36,2.055278e-32,1.065643,3.116198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,AAUGAACAUCACAAACAGCCU,1043.480079,9.692181,0.780177,12.423061,1.959203e-35,1.246902e-31,3.196929,1.558099,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
df_blast = pd.read_csv(blast_out,sep="\t", header=None)
df_blast.columns = ['qseqid','sseqid','qstart','qend','sstart','send','score','length','mismatch','gaps','gapopen','nident','pident','qlen','slen','qcovs']
print('initial:',len(df_blast.index))

initial: 94331


In [159]:
df_blast = df_blast[df_blast.length >= df_blast.qlen - 4]
print('len -4:',len(df_blast.index))

len -4: 42503


In [160]:
df_res = pd.merge(df_deg, df_blast[['qseqid','sseqid']], left_on='Gene', right_on='qseqid', how='left')
df_res = df_res.drop(['qseqid'], axis=1)
df_res.rename(columns={'sseqid': 'miRBASE'}, inplace=True)
df_res = df_res.replace(np.nan, '', regex=True)
print(len(df_res.index))
df_res.head(2)

56505


,Unnamed: 0,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,...,MIRNA,PhaseScore,Short,Long,20,21,22,23,24,miRBASE
0,1,UUGUACCAACUGUAAACUCGU,2442.153702,11.330772,0.881097,12.859849,7.572419e-38,1.445802e-33,2.131286,1.558099,...,,,,,,,,,,
1,2,AGAUUCUGUUGAACUCGUUCUC,1771.326168,10.859947,0.862006,12.598466,2.152912e-36,2.055278e-32,1.065643,3.116198,...,,,,,,,,,,


In [161]:
rules = {'MajorRNA':'first','log2FoldChange':'first','miRBASE':','.join}
df_res = df_res.groupby('Gene',as_index=False).agg(rules)

In [162]:
df_res.to_csv(data_dir + '/res/sRNA_DEG_mirbase.csv', sep='\t', index=None)